# Computational Finance
## Series 6
Tientso Ning

In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

In [2]:
#read data
data = pd.read_table("./data.dat", header=None, names=["timestamp", "bid", "ask"])
data["timestamp"] = data["timestamp"].apply(lambda x : time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(x)))

#trim data
data = data.head(np.around(len(data)*0.3).astype(int))

#pre-processing removal
data = data[data["timestamp"] > "2012-01-01 00:00:00"]

#process data
data.timestamp = pd.DatetimeIndex(data.timestamp)
data.set_index('timestamp', inplace=True)

In [3]:
#15 minute intervals
data = data.resample('15min').mean()

In [4]:
#TWAP is just the values we have at each 15 minute interval
data['TWAP'] = data['bid']

#VWAP calculations
data['volume'] = data.index.value_counts()
data['cumulative.volume'] = data['volume'].cumsum()
data['tp.v'] = data['bid']*data['volume']
data['cumulative.tp.v'] = data['tp.v'].cumsum()
data['VWAP'] = data['cumulative.tp.v']/data['cumulative.volume']

In [178]:
def ABPE (ts, delta):
    
    L = len(ts)
    direction = 1
    x = 0
    dc = 1
    pos = []
    neg = []
    
    for i in range (1, L): #cycle through the ts
        
        if np.abs(ts[x]-ts[i]) > delta*ts[x]: #scaling law
            
            #calculate directional change/overshoot
            if np.sign(ts[x]-ts[i]) != np.sign(direction):
                direction = direction * -1 #change direction
                dc += 1
                if np.sign(ts[x]-ts[i]) == 1:
                    neg.append(ts[i])
                elif np.sign(ts[x]-ts[i]) == -1:
                    pos.append(ts[i])
            else:
                #overshoot
                continue
                
            x = i #set new
        
    return dc, pos, neg

In [209]:
#sample 20 different time blocks
for i in range(20):
    x = np.random.randint(0,len(data))
    print(data[x:x+12][['TWAP','VWAP']])
    
    print("---")
    
    twap_price = np.sum(data[x:x+12][['TWAP']]*1000000)[0]
    twap_dec_price = (data.iloc[x][['TWAP']]*12000000)[0]
    print("TWAP Price: ", twap_price)
    print("Decision Price: ", twap_dec_price) 
    print("Difference: ", twap_price-twap_dec_price)
    
    print("---")
    
    vwap_price = np.sum(data[x:x+12][['VWAP']]*1000000)[0]
    vwap_dec_price = (data.iloc[x][['VWAP']]*12000000)[0]
    print("VWAP Price: ", vwap_price)
    print("TWAP - VWAP: ", twap_price-vwap_price)
    
    print("---")
    
    dcs, pos_prices, neg_prices = ABPE(data.TWAP.values, 0.0001)
    abpe_price = np.sum(np.multiply(neg_prices[0:12],1000000))
    print("ABPE Price: ", abpe_price)
    print("ABPE - TWAP: ", abpe_price-twap_price)
    print("ABPE - VWAP: ", abpe_price-vwap_price)
    
    print("===")

                         TWAP      VWAP
timestamp                              
2012-01-10 03:00:00  1.277091  1.009339
2012-01-10 03:15:00  1.277213  1.009670
2012-01-10 03:30:00  1.277662  1.010000
2012-01-10 03:45:00  1.277791  1.010329
2012-01-10 04:00:00  1.278843  1.010659
2012-01-10 04:15:00  1.278769  1.010988
2012-01-10 04:30:00  1.278842  1.011316
2012-01-10 04:45:00  1.278670  1.011643
2012-01-10 05:00:00  1.279054  1.011970
2012-01-10 05:15:00  1.279073  1.012296
2012-01-10 05:30:00  1.279284  1.012622
2012-01-10 05:45:00  1.278878  1.012946
---
TWAP Price:  15341171.095654506
Decision Price:  15325088.971962603
Difference:  16082.123691903427
---
VWAP Price:  12133779.856376154
TWAP - VWAP:  3207391.239278352
---
ABPE Price:  15514682.344888886
ABPE - TWAP:  173511.2492343802
ABPE - VWAP:  3380902.488512732
===
                         TWAP      VWAP
timestamp                              
2012-01-12 17:30:00  1.282547  1.071995
2012-01-12 17:45:00  1.283117  1.072194
2012

ABPE Price:  15514682.344888886
ABPE - TWAP:  318260.7493137326
ABPE - VWAP:  3920381.050653355
===
                         TWAP      VWAP
timestamp                              
2012-01-11 09:15:00  1.276508  1.044146
2012-01-11 09:30:00  1.275741  1.044395
2012-01-11 09:45:00  1.275815  1.044643
2012-01-11 10:00:00  1.275581  1.044890
2012-01-11 10:15:00  1.271872  1.045133
2012-01-11 10:30:00  1.270297  1.045373
2012-01-11 10:45:00  1.270201  1.045613
2012-01-11 11:00:00  1.269086  1.045851
2012-01-11 11:15:00  1.270340  1.046091
2012-01-11 11:30:00  1.270449  1.046329
2012-01-11 11:45:00  1.271298  1.046568
2012-01-11 12:00:00  1.272120  1.046808
---
TWAP Price:  15269306.088006658
Decision Price:  15318092.934782587
Difference:  -48786.84677592851
---
VWAP Price:  12545839.77198018
TWAP - VWAP:  2723466.316026479
---
ABPE Price:  15514682.344888886
ABPE - TWAP:  245376.25688222796
ABPE - VWAP:  2968842.572908707
===
                         TWAP      VWAP
timestamp               

Here, we can see that for the most part, TWAP trading algorithm performs better than the decision price made in one purchase at time `t`. However, VWAP prices are overall better than TWAP prices, and the Algorithm Based on Price Evolution tends to beat VWAP when it also beats the TWAP prices. This is likey due to the fact that our implementation of ABPE is based on the TWAP prices.